<a href="https://colab.research.google.com/github/LeeJeongWoon-Workout/Deep_Learning_Pytorch/blob/main/Kaggle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from torchvision import datasets,transforms
from torch.utils.data import Dataset,DataLoader

In [ ]:
from google.colab import drive 


drive.mount('/content/gdrive/')


Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [ ]:
sample=pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/otto/sampleSubmission.csv')
train_pd=pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/otto/train.csv')
test_pd=pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/otto/test.csv')

In [ ]:
class OttoDataset(Dataset):
    def __init__(self):
        xy = np.loadtxt('/content/gdrive/MyDrive/Colab Notebooks/otto/train.csv',delimiter=',',skiprows = 1, usecols = np.arange(1,94))
        df = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/otto/train.csv', sep = ',')
        df['target'] =  df['target'].map({'Class_1': 1, 'Class_2': 2,
                                          'Class_3': 3, 'Class_4': 4,
                                          'Class_5': 5, 'Class_6': 6,
                                          'Class_7': 7, 'Class_8': 8,
                                          'Class_9': 9})
        df['target'] = df['target'].astype('float64')
        self.len = xy.shape[0]
        self.x_data = torch.from_numpy(xy[:,:])
        self.x_data=self.x_data
        self.y_data = torch.tensor(df['target'].values)
        self.y_data=self.y_data

    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]

    def __len__(self):
        return self.len

dataset = OttoDataset()
train_loader = DataLoader(dataset=dataset,
                          batch_size=32,
                          shuffle=True,
                          num_workers=2)  

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        self.l1 = nn.Linear(93, 46)
        self.l2 = nn.Linear(46,18)
        self.l3 = nn.Linear(18,9)
    def forward(self,x):
        x = F.relu(self.l1(x))
        x = F.relu(self.l2(x))
        x=F.softmax(self.l3(x),dim=1)
        return self.l3(x)

model = Net()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)
model=model.cuda()

In [ ]:
import time
device = 'cuda' if torch.cuda.is_available() else 'cpu'
def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data.float())
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 10 == 0:
            print('Train Epoch: {} | Batch Status: {}/{} ({:.0f}%) | Loss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))





if __name__ == '__main__':
    since = time.time()
    for epoch in range(1, 10):
        epoch_start = time.time()
        train(epoch)
        m, s = divmod(time.time() - epoch_start, 60)
        print(f'Training time: {m:.0f}m {s:.0f}s')


    m, s = divmod(time.time() - since, 60)
    print(f'Total Time: {m:.0f}m {s:.0f}s\nModel was trained on {device}!')

RuntimeError: ignored

In [ ]:
def test():
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        data, target = data.to(device), target.to(device)
        output = model(data)
        # sum up batch loss
        test_loss += criterion(output, target).item()
        # get the index of the max
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()

    test_loss /= len(test_loader.dataset)
    print(f'===========================\nTest set: Average loss: {test_loss:.4f}, Accuracy: {correct}/{len(test_loader.dataset)} '
          f'({100. * correct / len(test_loader.dataset):.0f}%)')